In [4]:
import numpy as np
import os
import pandas as pd
from osmgraph import GraphFromBbox, GraphFromHmlFile, GraphFromGdfs
from spaitalShape import Point, OdPair, Box
from edgeGdfPreprocessing import edgePreprocessing
import osmnx as ox
import math
import time

# Profiling: python -m cProfile -o profile.pstats ecoRouting.py
# Visualize profile: snakeviz profile.pstats

# Profiling: python -m cProfile -o profile.pstats ecoRouting.py
# Visualize profile: snakeviz profile.pstats


class LocationRequest:
    def __init__(self):
        # Murphy Depot
        # (longitude, latitude)
        self.origin = Point(-93.2219, 44.979)
        # test
        #self.origin = Point(-93.4254, 44.7888)
        #self.origin = Point(-93.470167, 44.799720)
        #origin = Point(-93.2466, 44.8959)
        # Shakopee East (Depot):
        self.destination = Point(-93.4620, 44.7903)
        #self.destination = Point(-93.230358, 44.973583)

        self.odPair = OdPair(self.origin, self.destination)
        self.boundingBox = Box(-93.4975, -93.1850, 44.7458, 45.0045)
        self.mass = 23000
        # 9 am
        self.timeOfTheDay = 9
        # Monday
        self.dayOfTheWeek = 1


In [2]:
def testfunction(edgesGdf):
    from window import Window
    w = Window(61851, 61877, 61893, 33971)
    print(w.extractFeatures(edgesGdf))
    return

def extractGraphOf(boundingBox):
    folderOfGraph = r'GraphDataInBbox'
    if os.path.exists(folderOfGraph):
        print("reloading graph..")
        osmGraph = GraphFromHmlFile(os.path.join(folderOfGraph, 'osmGraph.graphml'))
    else:
        print("downloading graph..")
        osmGraph = GraphFromBbox(boundingBox)
        osmGraph.saveHmlTo(folderOfGraph)
    # fig, ax = ox.plot_graph(osmGraph.graph)
    return osmGraph


def extractElevation(nodes, edges):
    extractNodesElevation(nodes)
    extractEdgesElevation(nodes, edges)



def extractNodesElevation(nodes):
    nodesElevation = pd.read_csv("nodesWithElevation.csv", index_col=0)
    nodes['indexId'] = nodes.index
    nodes['elevation'] = nodes.apply(lambda x: nodesElevation.loc[x['indexId'], 'MeanElevation'], axis=1)



def extractEdgesElevation(nodesWithElevation, edges):
    edges['uElevation'] = edges['u'].apply(lambda x: nodesWithElevation.loc[x,'elevation'])
    edges['vElevation'] = edges['v'].apply(lambda x: nodesWithElevation.loc[x,'elevation'])


def extractGraphInMurphy(nodes, edges):
    edgesInMurphy = extractEdgesInMurphy(edges)
    graphInMurphy = GraphFromGdfs(nodes, edgesInMurphy)
    return graphInMurphy


def extractEdgesInMurphy(edges):
    edges['uvPair'] = edges.apply(lambda x: (x.u, x.v), axis=1)
    segmentElevationChange = np.load('statistical data/segmentElevationChange.npy', allow_pickle=True).item()
    edges['isInMurphy'] = edges.uvPair.apply(lambda x: x in segmentElevationChange)
    return edges[edges.isInMurphy]


def findShortestPath(osmGraph, localRequest):
    shortestPath = osmGraph.shortestPath(localRequest)
    print("shortestPath:", shortestPath)
    # ox.plot_graph(osmGraph)
    #osmGraph.plotPath(shortestPath, "shortest route.pdf")
    return shortestPath


def findEcoPathAndCalEnergy(osmGraph, localRequest):
    ecoPath, ecoEnergy , ecoEdgePath = osmGraph.ecoPath(localRequest)
    print("ecoPath:", ecoPath, "ecoEnergy:", ecoEnergy, ecoEdgePath)
    #osmGraph.plotPath(ecoPath, "eco route.pdf")
    return ecoPath, ecoEnergy,  ecoEdgePath


def findFastestPathAndCalTime(osmGraph, localRequest):

    fastestPath, shortestTime, fastEdgePath = osmGraph.fastestPath(localRequest)
    print("fastestPath:", fastestPath, "shortestTime:", shortestTime, fastEdgePath)
    #osmGraph.plotPath(fastestPath,"fastest route.pdf")
    return fastestPath, shortestTime, fastEdgePath


def nodePathTOEdgePath(nodePath, edgesGdf):
    edgePath = []
    for i, OdPair in enumerate(zip(nodePath[:-1], nodePath[1:])):
        segmentId = edgesGdf[edgesGdf['odPair'] == OdPair].index[0]
        edgePath.append(segmentId)
    return edgePath



def calAndPrintPathAttributes(osmGraph, edgePath, pathname):
    numberOfSegments = len(edgePath)
    length = osmGraph.totalLength(edgePath)
    energy = osmGraph.totalEnergy(edgePath)
    time = osmGraph.totalTime(edgePath)
    print(pathname+":"+f"{numberOfSegments} segments, {length} meters, {energy} liters, {time} seconds")
    return


In [5]:
locationRequest = LocationRequest()
osmGraphInBbox = extractGraphOf(locationRequest.boundingBox)
nodes, edges = osmGraphInBbox.graphToGdfs()
extractElevation(nodes, edges)
edges = edgePreprocessing(nodes, edges, locationRequest)
graphWithElevation = GraphFromGdfs(nodes, edges)
graphWithElevation.removeIsolateNodes()

reloading graph..


In [7]:
shortestNodePath =[301297715, 244110319, 34638990, 301295780, 33749238, 1674139069, 1641850667, 244111192, 1025799475, 1817826566, 33419919, 598048989, 598260853, 598260880, 598037615, 598037808, 33741160, 33465717, 34561215, 596664549, 596664559, 595906980, 595898782, 595955534, 595839400, 6928916280, 595819785, 595434003, 245812803, 740355099, 33308314, 476913479, 476914885, 33382792, 476914946, 34509163, 476916370, 34383306, 33750688, 34392416, 34397391, 34397632, 34188179, 34163786, 33464056, 33296901, 33316298, 33316299, 1355644211, 430586497, 34638790, 2939829463, 414212637, 476944911, 476944926, 476948454, 476948459, 6960948421, 678979183, 729441988, 729441987, 729441984, 561037952, 678980591, 678980588, 699924126, 6141477756, 678254900, 678254898, 678254896, 33457336, 678254893, 2472823034, 678254891, 678254888, 700682568, 700682567, 678254885, 33318883, 8227916672, 8493346868, 560964732, 678254883, 560655152, 564231574, 568935618, 593993693, 593993698, 700682565, 593993564, 595487079,34173817, 34644805, 33418566, 33541907, 33542034, 34384209, 33297585, 33749182, 34372961, 33525925, 33846344, 34397014, 34573439, 1912414860, 34543950, 1912414844, 33802895, 491892734, 34188784, 1912414775, 2016989359, 2016989354, 33806396, 5617149278, 34504806, 5617149285, 34186314, 33297281, 33352515, 33329222, 34668233, 976348692, 976348635, 976348672, 976348726, 976348722, 976348734, 976348621, 5402587238, 33419363, 33297843, 33297842, 33376173, 33371571, 5402587232, 5402587223, 1208071476, 1208071483, 470842396, 34653054, 33744485, 33319857, 465534614, 465534616, 534950337, 539673804, 1541658049, 534950306, 1649997656, 534950310, 300218109, 1649997646, 188038057, 188038056, 8146751356]


In [8]:
shortestPath = nodePathTOEdgePath(shortestNodePath, edges)

In [9]:
calAndPrintPathAttributes(graphWithElevation, shortestPath, "shortestPath")

shortestPath:155 segments, 37434.512 meters, 8.494811125099659 liters, 2706.8516367673874 seconds


In [10]:
ecoRoute = [301297715, 244110319, 34638990, 301297739, 244109286, 33802423, 34191216, 34178512, 34161335, 34177358, 34161335, 34638023, 33307879, 33307818, 246659838, 34511373, 34502016, 1849073245, 803066564, 33362637, 33362622, 803066706, 803066564, 33362637, 33362622, 267078135, 1883270850, 595928255, 595911061, 595955817, 595898810, 2498997791, 6876304186, 6876426598, 6876426587, 6878107187, 6876303755, 770727059, 1685304170, 1685304139, 770726730, 770928383, 770741788, 1001588962, 1001589148, 1001588927, 1001589024, 34498025, 1001589020, 1001588885, 560639812, 33328314, 486972580, 576885338, 486972620, 563387811, 563387842, 486972733, 593993633, 476751858, 476751860, 476751861, 486973044, 476272946, 476751734, 1522703178, 476272960, 497309536, 497309549, 33493793, 497309633, 33368310, 34638352, 470814180, 33373801, 470820638, 33640842, 246759348, 2015677691, 470825756, 470838251, 470838668, 34497967, 4210557790, 34653054, 33744485, 33319857, 465534614, 188045381, 470847865, 188062765, 469362905, 469329044, 469329045, 188055018, 539723216, 188055017, 188063558, 8146751356]

In [11]:
ecoEdgePath = nodePathTOEdgePath(ecoRoute, edges)

In [12]:
calAndPrintPathAttributes(graphWithElevation, ecoEdgePath, "ecoRoute")

ecoRoute:98 segments, 43447.779 meters, 4.119279943406582 liters, 1985.3674219846725 seconds


In [13]:
fastestPath = [301297715, 244110319, 34638990, 301295780, 301295799, 301297749, 33635653, 244111192, 1025799475, 1817826566, 33419919, 598048989, 598260853, 598260880, 598037615, 598037808, 596664549, 596664559, 595906980, 595898782, 595898810, 2498997791, 6876304186, 6878107187, 6876303755, 770727059, 1685304170, 1685304139, 770726730, 770928383, 770741788, 1001588998, 415333483, 415333485, 415168815, 415168795, 33750529, 34634773, 512438284, 503626094, 503629309, 503628607, 415167888, 2016984421,415167907, 33313247, 6788465524, 415167400, 1696968667, 2015677691, 470825756, 470838251, 470838668, 34497967, 4210557790, 34653054, 33744485, 33319857, 465534614, 465534616, 534950337, 539673804, 1541658049, 534950306, 1649997656, 534950310, 300218109, 1649997646, 188038057, 188038056, 8146751356] 

In [14]:
fastestEdgePath = nodePathTOEdgePath(fastestPath, edges)

In [15]:
calAndPrintPathAttributes(graphWithElevation, fastestEdgePath, "fastestPath")

fastestPath:70 segments, 38967.602 meters, 5.171727582812309 liters, 1684.5582154989243 seconds


In [16]:
import plotly.graph_objects as go
import plotly
# plot map matching results
def plotRoutes(routeList, network_gdf, colorLists, nameLists):
    directory = './results'
    colorLists = ['green', 'red', 'blue']
    if not os.path.exists(directory):
        os.makedirs(directory)
    edgeLongList = []
    edgeLatList = []
    for i in range(len(routeList)):
        route = routeList[i]
        long_edge = []
        lat_edge = []
        for j in route:
            e = network_gdf.loc[j]
            if 'geometry' in e:
                xs, ys = e['geometry'].xy
                z = list(zip(xs, ys))
                l1 = list(list(zip(*z))[0])
                l2 = list(list(zip(*z))[1])
                for k in range(len(l1)):
                    long_edge.append(l1[k])
                    lat_edge.append(l2[k])
        if i == 0:
            fig = go.Figure(go.Scattermapbox(
                name = nameLists[i],
                mode = "lines",
                lon = long_edge,
                lat = lat_edge,
                marker = {'size': 5, 'color':colorLists[i]},
                line = dict(width = 3, color = colorLists[i]))) 
        else:
            fig.add_trace(go.Scattermapbox(
                name = nameLists[i],
                mode = "lines",
                lon = long_edge,
                lat = lat_edge,
                marker = {'size': 5, 'color':colorLists[i]},
                line = dict(width = 3, color = colorLists[i]))) 
    # getting center for plots:
    lat_center = np.mean(lat_edge)
    long_center = np.mean(long_edge)
    zoom = 9.5
    # defining the layout using mapbox_style
    fig.update_layout(mapbox_style="stamen-terrain",
        mapbox_center_lat = 30, mapbox_center_lon=-80)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                    mapbox = {
                            'center': {'lat': lat_center, 
                            'lon': long_center},
                            'zoom': zoom})

    plotly.offline.plot(fig,filename = os.path.join(directory,'resultnewdropped.html'),auto_open=True)

In [17]:
plotRoutes([shortestPath, ecoEdgePath, fastestEdgePath],edges,['green', 'red', 'blue'],['shortest route','eco route','fastest route'])

In [1]:
def extractAllWindows(self, uToV):
    #uToV = self.getUToV()
    windowList, tempWindowStack, tempNodeIdStack = [], [], []  
    for i in uToV:
        listOfNodes = uToV[i]
        for edgeIdAndV in listOfNodes:
            edgeIdInGdf = edgeIdAndV[0]
            nextNodeId = edgeIdAndV[1]
            tempWindowStack.append(edgeIdInGdf)
            tempNodeIdStack.append(nextNodeId)
    while tempWindowStack:
        tempWindow = tempWindowStack.pop()
        tempNode = tempNodeIdStack.pop()
        if len(tempWindow) == 4:
            windowList.append(tempWindow)
            continue
        else:
            listOfNodes = uToV[tempNode]
            for edgeIdAndV in listOfNodes:
                edgeIdInGdf = edgeIdAndV[0]
                nextNodeId = edgeIdAndV[1]
                tempWindowStack.append(edgeIdInGdf)
                tempNodeIdStack.append(nextNodeId)
    return windowList

In [1]:
a = [1,1,1,1,3,4,[1,3]]

In [2]:
a[-1]

[1, 3]

In [3]:
a.pop()

[1, 3]

In [4]:
a

[1, 1, 1, 1, 3, 4]

In [1]:
a = [1,2]

In [2]:
b = []
b.append(a)

In [3]:
b

[[1, 2]]

In [4]:
a.pop()

2

In [5]:
b

[[1]]

In [1]:
a = [1]

In [2]:
list(a)

[1]

In [3]:
a = 1

In [1]:
a = dict()

In [2]:
a[1][2] = 3

KeyError: 1

In [3]:
a[(1,2)] = 3

In [4]:
a 

{(1, 2): 3}

In [1]:
import pickle

In [2]:
a = {3:4}

In [5]:
a

{3: 4}

In [6]:
filename = "a"

In [9]:
with open(filename+".pkl", "wb") as tf:
    pickle.dump(a, tf)

In [13]:
with open(filename+".pkl", "wb") as tf:
    loadDict = pickle.load(tf)

UnsupportedOperation: read

In [12]:
a = dict()
a[1] = (3,2)

In [13]:
b = a.values()

In [15]:
b

dict_values([(3, 2)])

In [16]:
list(b)

[(3, 2)]

In [4]:
a[1][2] =2

In [17]:
a = [1,3,45,]

In [24]:
a

[6, 3, 45]

In [25]:
b = a[0]

In [26]:
b

6

In [27]:
a[0] = 1

In [28]:
a

[1, 3, 45]

In [29]:
b

6

In [1]:
a =  2

In [4]:
print('the paper number is'+str(a))

the paper number is2


In [5]:
a = input()
b = int(a)
print("a",b,a,"end")

a 3 3 end


In [7]:
print("s", ':', sep='')

s:


In [8]:
print("s", ':')

s :
